<진화비용>
화재를 진압하기 위해 투입되는 자원(인력, 장비, 물자 등)  |
<간접비용>
지역 경제, 환경, 건강, 관광 산업 등에 미치는 피해

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 데이터 불러오기
df = pd.read_csv(r"/content/최종 데이터셋.csv")  # CSV 파일을 데이터프레임으로 로드

# 피해금액 쉼표 제거 후 숫자로 변환
df['피해금액'] = df['피해금액'].astype(str).str.replace(",", "").astype(float)  # 숫자로 변환

# 학습 데이터 분리
y = df['피해금액']  # 목표 변수 (타겟 값)
X = df.drop(columns=['피해금액','시도','시군구','읍면동리'])  # 입력 변수 (피해금액 제외)

# 훈련 데이터와 테스트 데이터 분리 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 설정 (하이퍼파라미터 조정 포함)
model = lgb.LGBMRegressor(
    n_estimators=500,        # 트리 개수 (500개)
    learning_rate=0.03,      # 학습률 (0.03)
    max_depth=6,             # 트리 최대 깊이 (6)
    num_leaves=32,           # 리프 노드 개수 (32)
    min_data_in_leaf=20,     # 최소 리프 노드 데이터 개수 (20)
    feature_fraction=0.8,    # 랜덤하게 선택된 피처의 비율 (80%)
    bagging_fraction=0.8,    # 샘플링 비율 (80%)
    bagging_freq=5,          # 배깅(샘플링) 빈도 (5회)
    lambda_l1=0.1,           # L1 정규화 (라쏘 회귀)
    lambda_l2=0.1,           # L2 정규화 (릿지 회귀)
    verbose=-1,              # 불필요한 로그 출력 방지
    random_state=42          # 랜덤 시드 고정 (재현 가능성 확보)
)

# 모델 학습
model.fit(
    X_train, y_train,                 # 학습 데이터
    eval_set=[(X_train, y_train), (X_test, y_test)],  # 검증 데이터 추가
    eval_names=['train', 'valid'],    # 평가 데이터 이름 설정
    eval_metric='mae',                # 평가 지표로 MAE(Mean Absolute Error) 사용
)

# 예측 수행
y_pred = model.predict(X_test)  # 테스트 데이터에 대한 예측값 생성
y_pred = y_pred.astype(int)  # 예측값을 정수형으로 변환 (소수점 제거)

# 모델 평가
mae = mean_absolute_error(y_test, y_pred)  # 평균 절대 오차 (MAE)
mse = mean_squared_error(y_test, y_pred)  # 평균 제곱 오차 (MSE)
rmse = mse ** 0.5  # 평균 제곱근 오차 (RMSE)
r2 = r2_score(y_test, y_pred)  # 결정 계수 (R², 모델의 설명력)

# 평가 지표 출력
print(f"MAE: {mae:.0f}")  # MAE 출력
print(f"MSE: {mse:.0f}")  # MSE 출력
print(f"RMSE: {rmse:.0f}")  # RMSE 출력
print(f"R2 Score: {r2:.4f}")  # R² 점수 출력


MAE: 1389074
MSE: 96286318060489
RMSE: 9812559
R2 Score: 0.9685
